# Load Model YAML

In [61]:
import yaml

MODEL_FILE = '/mnt/hdd01/hyeongdo/workspace/yolov9/models/detect/yolov9-e.yaml'
with open(MODEL_FILE, encoding='ascii', errors='ignore') as f:
    model_architecture = yaml.safe_load(f)

print(type(model_architecture))
print(model_architecture.keys())


<class 'dict'>
dict_keys(['nc', 'depth_multiple', 'width_multiple', 'anchors', 'backbone', 'head'])


# Model Load

In [6]:
import os
root=os.path.dirname(os.getcwd())
# print(root)
import sys
sys.path.append(root)

import torch

from models.yolo import Model
from models.experimental import attempt_load

In [55]:
weights = '../experiments/gelan-e-1280-scratch-done/weights/gelan-e-1280-best-20240520.pt'
device = torch.device('cpu')
model = attempt_load(weights, device)
model.eval()
names = model.module.names if hasattr(model, 'module') else model.names # class names
print(names)
print(model.names)
# print(type(model.model))
# print(len(model.model))
# print(model.model[1].type)
# print(model.model[1])
target_layers = [1] # target layer's index from yaml file

# weight = torch.load(weights)
# model_names = weight['model'].names
# print(model_names)
# print(type(weight))
# print(weight['epoch'], weight['best_fitness'])

Fusing layers... 
gelan-e-sar summary: 690 layers, 57285011 parameters, 0 gradients, 188.6 GFLOPs


{0: 'item'}
{0: 'item'}


# Common

In [1]:
import torch
from models.common import DetectMultiBackend
from utils.general import check_img_size

모델 생성을 위한 변수값을 설정한다.

In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
weights = '/work/src/yolov9/weights/yolov9_c_motsynth_21epoch.pt'
dnn = False
data = '/work/src/yolov9/data/yolo-anatomy.yaml'
half = True
imgsz = 640

설정한 변수값을 적용해서 모델을 생성한다.

In [3]:
model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)

Fusing layers... 
yolov9-c summary: 604 layers, 50698278 parameters, 0 gradients, 236.6 GFLOPs


In [4]:
stride, pt, jit, engine = model.stride, model.pt, model.jit, model.engine

In [5]:
print(stride)
print(pt)
print(jit)
print(engine)

32
True
False
False


In [6]:
imgsz = check_img_size(imgsz, s=stride)
print(imgsz)

640


In [7]:
from utils.general import check_dataset

In [8]:
data = check_dataset(data)
print(data)

{'path': '/work/dataset', 'train': '/work/dataset/vr-drone-test/vr-drone-test.txt', 'val': '/work/dataset/vr-drone-test/vr-drone-test.txt', 'test': '/work/dataset/vr-drone-test/vr-drone-test.txt', 'names': {0: 'p'}, 'nc': 1}


In [9]:
model.eval()

DetectMultiBackend(
  (model): DetectionModel(
    (model): Sequential(
      (0): Silence()
      (1): Conv(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (2): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (3): RepNCSPELAN4(
        (cv1): Conv(
          (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv2): Sequential(
          (0): RepNCSP(
            (cv1): Conv(
              (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
              (act): SiLU(inplace=True)
            )
            (cv2): Conv(
              (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
              (act): SiLU(inplace=True)
            )
            (cv3): Conv(
              (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
             

In [10]:
nc = int(data['nc'])
print(nc)
iouv = torch.linspace(0.5, 0.95, 10, device=device)
print(iouv)
niou = iouv.numel()
print(niou)

1
tensor([0.50000, 0.55000, 0.60000, 0.65000, 0.70000, 0.75000, 0.80000, 0.85000, 0.90000, 0.95000], device='cuda:0')
10


In [11]:
model.warmup(imgsz=(1, 3, imgsz, imgsz))

In [12]:
pad, rect = 0.5, pt
print(pad)
print(rect)

0.5
True


In [13]:
task = 'test'

In [14]:
batch_size = 1
single_cls = False
workers = 2
min_items = 0

In [15]:
from utils.dataloaders import create_dataloader
from utils.general import colorstr

In [18]:
dataloader = create_dataloader(data[task],
                               imgsz,
                               batch_size,
                               stride,
                               single_cls,
                               pad=pad,
                               rect=rect,
                               workers=workers,
                               min_items=min_items,
                               prefix=colorstr(f'{task}: '))[0]

test: Scanning /work/dataset/vr-drone-test/vr-drone-test... 39 images, 0 backgrounds, 0 corrupt: 100%|██████████| 39/39 00:00
test: New cache created: /work/dataset/vr-drone-test/vr-drone-test.cache


In [21]:
from utils.general import (Profile)

In [22]:
dt = Profile(), Profile(), Profile()

In [23]:
dt[0]

In [25]:
cuda = device.type != 'cpu'
augment = False
save_hybrid = False

In [58]:
from utils.general import non_max_suppression

In [57]:
for batch_i, (im, targets, paths, shapes) in enumerate(dataloader):
    print(batch_i, targets.size(), paths, shapes)
    # targets[number of labels, 6] label은 1:6까지가 유효
    # profiling start
    with dt[0]:
        if cuda:
            im = im.to(device, non_blocking=True)
            targets = targets.to(device)
        im = im.half() if half else im.float()
        im /= 255 # 0 - 255 to 0.0 - 1.0
        nb, _, height, width = im.shape # batch size, channel, height, width

    with dt[1]:
        preds = model(im, augment=augment) # [[torch.Tensor, torch.Tensor], [[torch.Tensor], [torch.Tensor], [torch.Tensor]]]
        # print(preds[1][1][0].shape)
    preds = preds[0][1]

    # NMS
    # label 좌표를 상대 좌표에서 이미지 절대 좌표로 변환
    targets[:, 2:] *= torch.tensor((width, height, width, height), device=device)
    # label+predictions을 같이 저장
    lb = [targets[targets[:,0]==i, 1:] for i in range(nb)] if save_hybrid else []

    with dt[2]:
        preds = non_max_suppression(preds,
                                    conf_thres,
                                    iou_thres,
                                    labels=lb,
                                    multi_label=True,
                                    agnostic=single_cls,
                                    max_det=max_det)

0 torch.Size([12, 6]) ['/work/dataset/vr-drone-test/images/Crop_0039_030m_30c_4_4_00404.jpg'] [[[1512, 2688], [[0.23809523809523808, 0.23809523809523808], [16.0, 12.0]]]]
1 torch.Size([9, 6]) ['/work/dataset/vr-drone-test/images/Crop_0039_030m_30c_4_4_00405.jpg'] [[[1512, 2688], [[0.23809523809523808, 0.23809523809523808], [16.0, 12.0]]]]
2 torch.Size([12, 6]) ['/work/dataset/vr-drone-test/images/Crop_0039_030m_30c_4_4_00406.jpg'] [[[1512, 2688], [[0.23809523809523808, 0.23809523809523808], [16.0, 12.0]]]]
3 torch.Size([16, 6]) ['/work/dataset/vr-drone-test/images/Crop_0040_030m_30c_5_5_00413.jpg'] [[[1512, 2688], [[0.23809523809523808, 0.23809523809523808], [16.0, 12.0]]]]
4 torch.Size([15, 6]) ['/work/dataset/vr-drone-test/images/Crop_0040_030m_30c_5_5_00414.jpg'] [[[1512, 2688], [[0.23809523809523808, 0.23809523809523808], [16.0, 12.0]]]]
5 torch.Size([7, 6]) ['/work/dataset/vr-drone-test/images/Crop_0040_030m_30c_5_5_00415.jpg'] [[[1512, 2688], [[0.23809523809523808, 0.238095238095

In [2]:
import glob

In [11]:
img_files = glob.glob('/work/dataset/VR-DRONE-v1.0.0.test/20221207/**/*', recursive=True)

In [12]:
print(len(img_files))

3610


In [13]:
img_files

['/work/dataset/VR-DRONE-v1.0.0.test/20221207/test_1class_all.txt',
 '/work/dataset/VR-DRONE-v1.0.0.test/20221207/1Class',
 '/work/dataset/VR-DRONE-v1.0.0.test/20221207/test_1class_all.cache',
 '/work/dataset/VR-DRONE-v1.0.0.test/20221207/2Class',
 '/work/dataset/VR-DRONE-v1.0.0.test/20221207/test_2class_all.txt',
 '/work/dataset/VR-DRONE-v1.0.0.test/20221207/1Class/10m',
 '/work/dataset/VR-DRONE-v1.0.0.test/20221207/1Class/15m',
 '/work/dataset/VR-DRONE-v1.0.0.test/20221207/1Class/50m',
 '/work/dataset/VR-DRONE-v1.0.0.test/20221207/1Class/20m',
 '/work/dataset/VR-DRONE-v1.0.0.test/20221207/1Class/40m',
 '/work/dataset/VR-DRONE-v1.0.0.test/20221207/1Class/60m',
 '/work/dataset/VR-DRONE-v1.0.0.test/20221207/1Class/100m',
 '/work/dataset/VR-DRONE-v1.0.0.test/20221207/1Class/80m',
 '/work/dataset/VR-DRONE-v1.0.0.test/20221207/1Class/30m',
 '/work/dataset/VR-DRONE-v1.0.0.test/20221207/1Class/10m/60c',
 '/work/dataset/VR-DRONE-v1.0.0.test/20221207/1Class/10m/30c',
 '/work/dataset/VR-DRONE-v

In [1]:
model_yaml_file = '/work/src/yolov9/models/detect/gelan-c.yaml'

In [3]:
import yaml

with open(model_yaml_file, encoding='ascii', errors='ignore') as f:
    model_yaml = yaml.safe_load(f)
print(model_yaml)

{'nc': 80, 'depth_multiple': 1.0, 'width_multiple': 1.0, 'anchors': 3, 'backbone': [[-1, 1, 'Conv', [64, 3, 2]], [-1, 1, 'Conv', [128, 3, 2]], [-1, 1, 'RepNCSPELAN4', [256, 128, 64, 1]], [-1, 1, 'ADown', [256]], [-1, 1, 'RepNCSPELAN4', [512, 256, 128, 1]], [-1, 1, 'ADown', [512]], [-1, 1, 'RepNCSPELAN4', [512, 512, 256, 1]], [-1, 1, 'ADown', [512]], [-1, 1, 'RepNCSPELAN4', [512, 512, 256, 1]]], 'head': [[-1, 1, 'SPPELAN', [512, 256]], [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']], [[-1, 6], 1, 'Concat', [1]], [-1, 1, 'RepNCSPELAN4', [512, 512, 256, 1]], [-1, 1, 'nn.Upsample', ['None', 2, 'nearest']], [[-1, 4], 1, 'Concat', [1]], [-1, 1, 'RepNCSPELAN4', [256, 256, 128, 1]], [-1, 1, 'ADown', [256]], [[-1, 12], 1, 'Concat', [1]], [-1, 1, 'RepNCSPELAN4', [512, 512, 256, 1]], [-1, 1, 'ADown', [512]], [[-1, 9], 1, 'Concat', [1]], [-1, 1, 'RepNCSPELAN4', [512, 512, 256, 1]], [[15, 18, 21], 1, 'DDetect', ['nc']]]}


In [6]:
model_yaml.get('ch', 5)
d = model_yaml

In [12]:
act = d['activation'] = d.get('activation', 'Test')
print(d.get('activation'), act)

Test Test


In [14]:
class Test:
    def __init__(self) -> None:
        pass

eval('Test')

__main__.Test

In [16]:
eval('64')

64